# Create atmos iron deposition for regional MITgcm based on GLODAPv2 and Johnson and Meskhidze (2013)
    - The data is based on 2009 meteorology

In [1]:
import numpy as np
import xarray as xr
import xmitgcm as xm
import os
from scipy.interpolate import RectBivariateSpline
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
srcdir='./JM2013/'

In [3]:
# define new grid
runcase = 'NorthPacific.lonlat2x2.42lev'
rname = 'NorthPacific'
xrange = [131, 249]  # zonal extent
yrange = [5, 62]  # meridional extent
n = [60, 40]   # number of grid points

In [4]:
#define a new grid coordinates
xn = np.linspace(xrange[0], xrange[1], n[0])
yn = np.linspace(yrange[0], yrange[1], n[1])
zn=np.array([-5.0000e+00, -1.5500e+01, -2.7000e+01, -3.9500e+01, -5.3000e+01,
       -6.8000e+01, -8.5000e+01, -1.0400e+02, -1.2550e+02, -1.5000e+02,
       -1.7750e+02, -2.0850e+02, -2.4350e+02, -2.8300e+02, -3.2800e+02,
       -3.7950e+02, -4.3850e+02, -5.0600e+02, -5.8300e+02, -6.7100e+02,
       -7.7200e+02, -8.8800e+02, -1.0210e+03, -1.1735e+03, -1.3485e+03,
       -1.5495e+03, -1.7805e+03, -2.0460e+03, -2.3190e+03, -2.5750e+03,
       -2.8250e+03, -3.0750e+03, -3.3250e+03, -3.5750e+03, -3.8250e+03,
       -4.0750e+03, -4.3250e+03, -4.5750e+03, -4.8250e+03, -5.0750e+03,
       -5.3250e+03, -5.5750e+03], dtype=float) * -1
#
# print the values of x0 and y0
x0 = xn[0]-(xn[1]-xn[0])/2.
y0 = yn[0]-(yn[1]-yn[0])/2.
print('x0='+str(x0))
print('y0='+str(y0))
#
# print the number of grid points in x, y and z
print('Nx='+str(xn.size))
print('Ny='+str(yn.size))
print('Nz='+str(zn.size))

x0=130.0
y0=4.269230769230769
Nx=60
Ny=40
Nz=42


In [5]:
# Load latitude and longitude data
LAT_center = np.loadtxt(srcdir+'LAT_center.ascii')
LONG_center = np.loadtxt(srcdir+'LONG_center.ascii')
y = LAT_center[:,0]
x = LONG_center[0,:]
Nx=x.size
Ny=y.size

In [6]:
dif=np.zeros((12,Ny,Nx))
solfe=np.zeros((12,Ny,Nx))
totfe1=np.zeros((12,Ny,Nx))
ym=['2009_03', '2009_04', '2009_05', '2009_06', 
    '2009_07', '2009_08', '2009_09', '2009_10', 
    '2009_11', '2009_12', '2010_01', '2010_02']
#
# Iterate over the months
for i, ym_i in enumerate(ym):
    # Load DIF_percent data for the current month
    fn = f'DIF_percent_{ym_i}.ascii'
    dif[i,:,:] = np.loadtxt(srcdir+fn)
    
    # Load SOLFE_DEP_ug_m2_month data for the current month
    fn = f'SOLFE_DEP_ug_m2_month_{ym_i}.ascii'
    solfe[i,:,:] = np.loadtxt(srcdir+fn)
    
    # Calculate totfe1 by dividing solfe by dif and multiplying by 100
    totfe1[i,:,:] = solfe[i,:,:] / dif[i,:,:] * 100

In [7]:
tmp1=np.zeros((12,Ny,Nx))
mo = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1, 2]
#
xd=np.roll(x,int(Nx/2))
xd=np.where(xd<0,xd+360,xd)
yd=y
month=np.arange(1,13,1)
#
for i in range(12):
    tmp = solfe[i, :, :] / 86400 / 30.4 / 55.845 * 1e-6 
    # converted to mol/m2/sec
    tmp1[mo[i]-1,:,:]=np.roll(tmp,int(Nx/2),axis=1)

#plt.pcolormesh(xd,yd,np.log10(tmp1[0,:,:]))

In [8]:
# Create a DataArray using tmp1 data
da = xr.DataArray(data=tmp1, name='solfedep', dims=['month', 'lat', 'lon'], coords={'month': month, 'lat': yd, 'lon': xd})

# Interpolate the DataArray to new latitude and longitude coordinates
sfed = da.interp(lat=yn, lon=xn).to_numpy()

# Write the sfed array to a binary file
xm.utils.write_to_binary(sfed.flatten(), 'solFe_dep_JM2013.bin')
